In [1]:
import sys
import os
from os import pardir, getcwd
from os.path import join, abspath
PARENT_DIRECTORY = abspath(join(getcwd(), pardir))
sys.path.insert(0, PARENT_DIRECTORY)

import warnings
import sklearn.exceptions
import talos as ta
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)


from definitions import TALOS_DIR
from app.preprocessing import load_dataset,preprocess_data
from app.models import load_model


Using TensorFlow backend.


In [2]:
data = load_dataset(load_from_pickle=True, input_ins='as_tf_idf')
Classes = list(data['tags'].value_counts().index)
Nclasses = len(Classes)
print(data['tags'].value_counts())

,post,tags
0,what is causing this behavior in our c# datet...,c#
1,have dynamic html load as if it was in an ifra...,asp.net
2,how to convert a float value in to min:sec i ...,objective-c
3,.net framework 4 redistributable just wonderi...,.net
4,trying to calculate and print the mean and its...,python
5,how to give alias name for my website i have ...,asp.net
6,window.open() returns null in angularjs it wo...,angularjs
7,identifying server timeout quickly in iphone ...,iphone
8,unknown method key error in rails 2.3.8 unit ...,ruby-on-rails
9,from the include how to show and hide the con...,angularjs


In [3]:
# 70% Train & 10% HeldOut & 20% Test
model_data_tf = preprocess_data(data, 'tags', 'post',
                                input_ins='as_tf_idf',
                                cv_split_full=0.2,
                                cv_split_dev=0.125)



MemoryError: 

In [3]:
'''Use the test filename if ypu don't want to delete
    # your current logs from the previous run.'''
TALOS_TF_LOG_FILENAME = 'talos_tf_log'
talos_tf_log_pathname = os.path.join(TALOS_DIR, TALOS_TF_LOG_FILENAME)

###### Production configuration
tf_idf_model_config = {
    'visualize_proccess': [False],
    'first_neuron': [64, 128],  # First Layer
    'activation': ['relu', 'tanh'],
    'dropout': [0.2, 0.6],
    'number_of_hidden_layers': [0, 1, 2],
    'shapes':['funnel'],
    'epochs': [4, 5, 7],
    'batch_size': [32, 64],
    'model_type': ['keras_tf_idf_model'],
}

history_model_tf_idf = ta.Scan(model_data_tf['x_train'],
                               model_data_tf['y_train'],
                               x_val=model_data_tf['x_train_dev'],
                               y_val=model_data_tf['y_train_dev'],
                               model=load_model,
                               params=tf_idf_model_config,
                               grid_downsample=0.2,
                               print_params=True,
                               seed=(123),
                               dataset_name=talos_tf_log_pathname
                               )


sql              2000
ruby-on-rails    2000
android          2000
angularjs        2000
asp.net          2000
c                2000
c#               2000
c++              2000
css              2000
html             2000
ios              2000
iphone           2000
java             2000
javascript       2000
jquery           2000
mysql            2000
objective-c      2000
php              2000
python           2000
.net             2000
Name: tags, dtype: int64
caus behavior c# datetim type <pre><code>[test] public void sadness() { var datetim = datetime.utcnow; assert.that(datetim is.equalto(datetime.parse(datetime.tostring()))); } </code></pre> fail <pre><code> expected: - - : : . was: - - : : . </code></pre> wish know happen behind scene tostring() etc caus behavior. edit see jon answer <pre><code>[test] public void newsadness() { var datetim = datetime.utcnow; assert.that(datetim is.equalto(datetime.parse(datetime.tostring( )))); } </code></pre> result <pre><code>expected: - - : : .

In [4]:
# 70% Train & 30% Test
# 70% Train-Dev % 30* Train-Dev 
model_data_sdr_tf = preprocess_data(data, 'tags', 'post',
                                 input_ins='as_tf_idf',
                                 cv_split_full=0.3,
                                 cv_split_dev=0.3,
                                 standarize=True
                                 )

In [5]:
TALOS_SDR_TF_LOG_FILENAME_TEST = 'talos_sdr_tf_log_test'
TALOS_SDR_TF_LOG_FILENAME = 'talos_sdr_tf_log'
talos_sdr_tf_log_pathname_test = os.path.join(DATA_DIR, TALOS_SDR_TF_LOG_FILENAME_TEST)
talos_sdr_tf_log_pathname = os.path.join(DATA_DIR, TALOS_SDR_TF_LOG_FILENAME)

##### Simplified configuration
sdr_tf_model_config = {
    'visualize_proccess': [True],
    'first_neuron': [256],
    'activation': ['relu'],
    'dropout': [0.5],
    'number_of_hidden_layers': [2],
    'epochs': [10],
    'batch_size': [32],
    'model_type': ['keras_embeddings_centroids_model'],
}

history_model_sdr_tf = ta.Scan(model_data_sdr_tf['x_train'],
                               model_data_sdr_tf['y_train'],
                               x_val=model_data_sdr_tf['x_train_dev'],
                               y_val=model_data_sdr_tf['y_train_dev'],
                               model=load_model,
                               params=sdr_tf_model_config,
                               grid_downsample=1,
                               print_params=True,
                               seed=(123),
                               dataset_name=talos_sdr_tf_log_pathname_test
                               )

  0%|          | 0/1 [00:00<?, ?it/s]

{'visualize_proccess': True, 'first_neuron': 256, 'activation': 'relu', 'dropout': 0.5, 'number_of_hidden_layers': 2, 'epochs': 10, 'batch_size': 32, 'model_type': 'keras_embeddings_centroids_model'}
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


100%|██████████| 1/1 [01:35<00:00, 95.76s/it]


In [6]:
# 70% Train & 30% Test
# 70% Train-Dev % 30* Train-Dev 
embeddings = load_embeddings(data, 'post', minimized=False)
model_data_sdr_ftc = preprocess_data(data, 'tags', 'post',
                                 input_ins='as_centroids',
                                 cv_split_full=0.3,
                                 cv_split_dev=0.3,
                                 embeddings=embeddings)


In [7]:
TALOS_SDR_FTC_LOG_FILENAME_TEST = 'talos_sdr_ftc_log_test'
TALOS_SDR_FTC_LOG_FILENAME = 'talos_sdr_ftc__log'
talos_sdr_ftc_log_pathname_test = os.path.join(DATA_DIR, TALOS_SDR_FTC_LOG_FILENAME_TEST)
talos_sdr_ftc_log_pathname = os.path.join(DATA_DIR, TALOS_SDR_FTC_LOG_FILENAME)

##### Simplified configuration
sdr_ftc_model_config = {
    'visualize_proccess': [True],
    'first_neuron': [256],
    'activation': ['relu'],
    'dropout': [0.5],
    'number_of_hidden_layers': [2],
    'epochs': [10],
    'batch_size': [32],
    'model_type': ['keras_embeddings_centroids_model'],
}

history_model_sdr_ftc = ta.Scan(model_data_sdr_ftc['x_train'],
                               model_data_sdr_ftc['y_train'],
                               x_val=model_data_sdr_ftc['x_train_dev'],
                               y_val=model_data_sdr_ftc['y_train_dev'],
                               model=load_model,
                               params=sdr_ftc_model_config,
                               grid_downsample=1,
                               print_params=True,
                               seed=(123),
                               dataset_name=talos_sdr_ftc_log_pathname_test
                               )

  0%|          | 0/1 [00:00<?, ?it/s]

{'visualize_proccess': True, 'first_neuron': 256, 'activation': 'relu', 'dropout': 0.5, 'number_of_hidden_layers': 2, 'epochs': 10, 'batch_size': 32, 'model_type': 'keras_embeddings_centroids_model'}


100%|██████████| 1/1 [00:20<00:00, 20.15s/it]


In [8]:
report_sdr_ftc = ta.Reporting(history_model_sdr_ftc)
best_model_idx = report_sdr_ftc.data['val_f1'].idxmax()
best_model_params = report_sdr_ftc.data.loc[best_model_idx].to_dict()
best_model_params

{'round_epochs': 10,
 'val_loss': 1.3911017615454537,
 'val_f1': 0.3213900066273553,
 'val_accuracy': 0.9599463871547154,
 'loss': 1.6155203059254861,
 'f1': 0.32981642442698383,
 'accuracy': 0.9563417912502679,
 'visualize_proccess': 'True',
 'first_neuron': 256,
 'activation': 'relu',
 'dropout': 0.5,
 'number_of_hidden_layers': 2,
 'epochs': 10,
 'batch_size': 32,
 'model_type': 'keras_embeddings_centroids_model'}